<a href="https://colab.research.google.com/github/kazukiminemura/rock_paper_scissors_colab/blob/main/Inference_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.8.0


In [27]:
import time
import numpy as np
import tensorflow_hub as hub # efficientnet model

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!cp '/content/drive/My Drive/efficientnet.h5' './efficientnet.h5'
!cp '/content/drive/My Drive/labels.txt' './labels.txt'
!ls

colab_camera.py    drive	    labels.txt	 sample_data
colab_camera.py.1  efficientnet.h5  __pycache__


In [29]:
# load model
keras_model = tf.keras.models.load_model("efficientnet.h5", custom_objects={'KerasLayer': hub.KerasLayer})
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              5330564   
                                                                 
 dense (Dense)               (None, 3)                 3003      
                                                                 
Total params: 5,333,567
Trainable params: 3,003
Non-trainable params: 5,330,564
_________________________________________________________________


In [30]:
labels = []
with open('labels.txt', 'r') as f:
  for line in f:
    labels.append(line.rstrip())
print(labels)

NUM_CLASSES = len(labels)

['choki', 'gu', 'pa']


In [31]:
# run demo
!wget https://raw.githubusercontent.com/karaage0703/karaage-ai-book/master/util/colab_camera.py
import colab_camera


--2022-04-03 17:23:29--  https://raw.githubusercontent.com/karaage0703/karaage-ai-book/master/util/colab_camera.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3036 (3.0K) [text/plain]
Saving to: ‘colab_camera.py.2’

colab_camera.py.2   100%[===================>]   2.96K  --.-KB/s    in 0s      

2022-04-03 17:23:29 (44.3 MB/s) - ‘colab_camera.py.2’ saved [3036/3036]



In [32]:
import cv2
import base64
import IPython
import numpy as np

from PIL import Image
from io import BytesIO
from google.colab import output

In [37]:
def run(img_str):
  # decode to image
  decimg = base64.b64decode(img_str.split(',')[1], validate=True)
  decimg = Image.open(BytesIO(decimg))
  decimg = np.array(decimg, dtype=np.uint8)
  decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

  # image classify
  img = cv2.resize(decimg, (64, 64))
  img = img_to_array(img)
  img = img / 255.0
  img = img[np.newaxis]
  img = np.asarray(img)

  preds = keras_model.predict(img)
  result = np.argmax(preds[0])

  # draw output
  out_img = decimg
  cv2.putText(out_img, labels[result], (10, 50), \
              cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
  
  # encode to string
  _, encimg = cv2.imencode(".jpg", out_img,
                           [int(cv2.IMWRITE_JPEG_QUALITY), 80])
  img_str = encimg.tobytes()
  img_str = ("data:image/jpeg;base64," +
             base64.b64encode(img_str).decode('utf-8'))
  
  return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)

In [38]:
colab_camera.use_cam()

<IPython.core.display.Javascript object>